In [17]:
import torch
from transformers import BertTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, random_split
from tqdm import tqdm
from tqdm import trange
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random 

In [2]:
def get_device():
    # Если в системе есть GPU ...
    if torch.cuda.is_available():
        # Тогда говорим PyTorch использовать GPU.
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(0))
    # Если нет GPU, то считаем на обычном процессоре ...
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device


device = get_device()

No GPU available, using the CPU instead.


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)



In [4]:
df_train = pd.read_csv('train.csv')

df_train.loc[(df_train.label == 'Good'), 'label'] = 1
df_train.loc[(df_train.label == 'Bad'), 'label'] = 0

df_train

,review_id,movie_id,text,label
0,0,0,\nСтарая поговорка гласит: «Лучшие рассказы — ...,1
1,1,0,"\nСамое сильное кино начала этого года, или ко...",1
2,2,0,\nДушевно. Когда противоположности встречаются...,1
3,3,0,"\nОб этом фильме я вообще ничего не знал, но н...",1
4,4,0,"\nКак правило, история людей, которые прикован...",1
...,...,...,...,...
16779,16779,189,"\n Если вам интересно, что получится, есл...",0
16780,16780,189,"\n Бывает так, собрался в кино, но на зар...",0
16781,16781,189,"\n Некоторым понравился этот фильм, некот...",0
16782,16782,189,"\n Когда я посмотрел некоторые отзывы, то...",0


In [6]:
texts = df_train['text']
labels = df_train['label']

input_ids, attention_masks = [], []

# Для всех предложений...
for text in tqdm(texts):
    encoded_dict = tokenizer.encode_plus(
        text,  # Текст для токенизации.
        add_special_tokens=True,  # Добавляем '[CLS]' и '[SEP]'
        max_length=256,  # Дополняем [PAD] или обрезаем текст до 64 токенов.
        pad_to_max_length=True,
        return_attention_mask=True,  # Возвращаем также attn. masks.
        return_tensors='pt',  # Возвращаем в виде тензоров pytorch.
    )

    # Добавляем токенизированное предложение в список
    input_ids.append(encoded_dict['input_ids'])
    # И добавляем attention mask в список
    attention_masks.append(encoded_dict['attention_mask'])

# Конвертируем списки в полноценные тензоры Pytorch.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Печатаем предложение с номером 0, его токены (теперь в виде номеров в словаре) и.т.д.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])
print('Attention masks:', attention_masks[0])
print('Labels:', labels[0])

  0%|                                                                                        | 0/16784 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
D:\jupyter\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnin

Original:  
Старая поговорка гласит: «Лучшие рассказы — это рассказы написанные самой жизнью». Режиссёрскому дуэту Оливье Накаша и Эрика Толедано удалось правильно рассказать трагикомедию основанную на реальной истории бедного молодого араба из французского гетто и парализованного бизнесмена, которых судьба связала неожиданной дружбой, несмотря на то, что в реальности их ничего не связывает, будь то социальный статус, или взгляды на понимание жизни. 

Очаровательная трагикомедия «Неприкасаемые» прежде всего имеет сердце и ум, полностью свободный от штампов о проявлении сострадания к инвалидам и прочим недееспособным людям. Фильм правильно позволяет им, в лице Филиппа, ощущать себя такими же практически полноценными людьми, как и все остальные, передавая всё это в юмористической форме, часто граничащей с обезоруживающей откровенностью. А происходит это на экране благодаря прежде всего увлекательной атмосфере, которую создаёт отличный дуэт Омара Сая, кажется имеющего в кармане свежую шут

In [8]:


# Объединяем все тренировочные данные в один TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Делаем разделение случайное разбиение 90% - тренировка 10% - валидация.

# Считаем число данных для тренировки и для валидации.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Разбиваем датасет с учетом посчитанного количества.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

15,105 training samples
1,679 validation samples


In [9]:
# DataLoader должен знать размер батча для тренировки мы задаем его здесь.
# Размер батча – это сколько текстов будет подаваться на сеть для вычисления градиентов
# Авторы BERT предлагают ставить его 16 или 32. 
batch_size = 32

# Создаем отдельные DataLoaders для наших тренировочного и валидационного наборов

# Для тренировки мы берем тексты в случайном порядке.
train_dataloader = DataLoader(
        train_dataset,  # Тренировочный набор данных.
        sampler = RandomSampler(train_dataset), # Выбираем батчи случайно
        batch_size = batch_size # Тренируем с таким размером батча.
)

# Для валидации порядок не важен, поэтому зачитываем их последовательно.
validation_dataloader = DataLoader(
        val_dataset, # Валидационный набор данных.
        sampler = SequentialSampler(val_dataset), # Выбираем батчи последовательно.
        batch_size = batch_size # Считаем качество модели с таким размером батча.
)

In [10]:
# Загружаем BertForSequenceClassification. Это предобученная модель BERT с одиночным полносвязным слоем для классификации
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Используем 12-слойную модель BERT, со словарем без регистра.
    num_labels = 2, # Количество выходных слоёв – 2 для бинарной классификации. Можно увеличить для мультиклассовой классификации.
    output_attentions = False, # Будет ли модель возвращать веса для attention-слоёв. В нашем случае нет.
    output_hidden_states = False, # Будет ли модель возвращать состояние всех скрытых слоёв. В нашем случае нет.
)

# Здесь мы говорим PyTorch что хотим тренировать модель на GPU.
if torch.cuda.is_available():
    model.cuda()

# Получаем все параметры модели как список кортежей и выводим сводную информацию по модели.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

D:\jupyter\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kibzi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', '

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [11]:
optimizer = AdamW(model.parameters(),
    lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
)


# Количество эпох для тренировки. Авторы BERT рекомендуют от 2 до 4.
# Мы выбираем 4, но увидим позже, что это приводит к оверфиту на тренировочные данные.
epochs = 1

# Общее число шагов тренировки равно [количество батчей] x [число эпох].
total_steps = len(train_dataloader) * epochs

# Создаем планировщик learning rate (LR). LR будет плавно уменьшаться в процессе тренировки
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

D:\jupyter\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
# Функция для расчёта точности. Сравниваются предсказания и реальная разметка к данным
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# На вход время в секундах и возвращается строка в формате hh:mm:ss
def format_time(elapsed):
    # Округляем до ближайшей секунды.
    elapsed_rounded = int(round((elapsed)))

    # Форматируем как hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [22]:
def train_step(device, model, train_dataloader, optimizer, scheduler):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_train_loss = 0
    # Переводим модель в режим тренировки.
    model.train()

    # Для каждого батча из тренировочных данных...
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Извлекаем все компоненты из полученного батча
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        # Очищаем все ранее посчитанные градиенты (это важно)
        model.zero_grad()
        # Выполняем прямой проход по данным
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        # Накапливаем тренировочную функцию потерь по всем батчам
        total_train_loss += output.loss
        # Выполняем обратное распространение ошибки что бы посчитать градиенты.
        output.loss.backward()
        # Ограничиваем максимальный размер градиента до 1.0. Это позволяет избежать проблемы "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Обновляем параметры модели используя рассчитанные градиенты с помощью выбранного оптимизатора и текущего learning rate.
        optimizer.step()
        # Обновляем learning rate.
        scheduler.step()

    # Считаем среднее значение функции потерь по всем батчам.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Сохраняем время тренировки одной эпохи.
    training_time = format_time(time.time() - t0)
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    return avg_train_loss, training_time

In [20]:
def validation_step(device, model, validation_dataloader):
    print("Running Validation...")
    t0 = time.time()
    # Переводим модель в режим evaluation – некоторые слои, например dropout ведут себя по другому.
    model.eval()

    # Переменные для подсчёта функции потерь и точности
    total_eval_accuracy = 0
    total_eval_loss = 0
    # Прогоняем все данные из валидации
    for batch in validation_dataloader:
        # Извлекаем все компоненты из полученного батча.
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

        # Говорим pytorch что нам не нужен вычислительный граф для подсчёта градиентов (всё будет работать намного быстрее)
        with torch.no_grad():
            # Прямой проход по нейронной сети и получение выходных значений.
            output = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        # Накапливаем значение функции потерь для валидации.
        total_eval_loss += output.loss

        # Переносим значения с GPU на CPU
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Считаем точность для отдельного батча с текстами и накапливаем значения.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Выводим точность для всех валидационных данных.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Считаем среднюю функцию потерь для всех батчей.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Измеряем как долго считалась валидация.
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    return avg_val_loss, avg_val_accuracy, validation_time

In [23]:
# В этой переменной сохраним всякую статистику по тренировке: точность, функцию цены (потерь) и время выполнения.
training_stats = []
# Переменная что бы измерить время всей тренировки.
total_t0 = time.time()

# Для каждой эпохи...
for epoch_i in range(0, epochs):
    # Запустить одну эпоху тренировки (следующий слайд) 
    avg_train_loss, training_time = train_step(device, model, train_dataloader, optimizer, scheduler)
    # Запустить валидацию что бы проверить качество модели на данном этапе (следующий слайд)
    avg_val_loss, avg_val_accuracy, validation_time = validation_step(device, model, validation_dataloader)

    # Сохраняем статистику тренировки на данной эпохе.
    training_stats.append(
        {
            'Epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validation Loss': avg_val_loss,
            'Validation Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("Training complete! Total training took {:} (hh:mm:ss)".format(format_time(time.time() - total_t0)))

======== Epoch 1 / 1 ========


KeyboardInterrupt: 

In [ ]:
test = pd.read_csv('test.csv')
sentences = test['text'].values

In [ ]:
input_ids = []
attention_masks = []

for text in tqdm(texts):
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        truncation=True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


In [ ]:
test_dataset = TensorDataset(input_ids, attention_masks)
test_dataloader = DataLoader(
        test_dataset,
        sampler = SequentialSampler(test_dataset),
        batch_size = 8
)